In [5]:
import Pkg
Pkg.add("Agents")
Pkg.add("CairoMakie")

  Installing known registries into `~/.julia`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Calculus ──────────────────── v0.5.1
   Installed IrrationalConstants ───────── v0.2.2
   Installed JLD2 ──────────────────────── v0.4.47
   Installed Scratch ───────────────────── v1.2.1
   Installed RealDot ───────────────────── v0.1.0
   Installed Crayons ───────────────────── v4.1.1
   Installed MixedStructTypes ──────────── v0.2.19
   Installed Rmath ─────────────────────── v0.7.1
   Installed XML2_jll ──────────────────── v2.12.6+0
   Installed QuickHeaps ────────────────── v0.1.2
   Installed ArnoldiMethod ─────────────── v0.4.0
   Installed DualNumbers ───────────────── v0.6.8
   Installed HypergeometricFunctions ───── v0.3.23
   Installed StreamSampling ────────────── v0.3.4
   Installed Preferences ───────────────── v1.4.3
   Installed StatsFuns ─────────────────── v1.3.1
   Installed JSON ──────────────────────── v0.21.4
   I

In [1]:
include("./predator_prey.jl")
CairoMakie.activate!() # hide

# To view our starting population, we can build an overview plot using [`abmplot`](@ref).
# We define the plotting details for the wolves and sheep:
offset(a) = a isa Sheep ? (-0.1, -0.1*rand()) : (+0.1, +0.1*rand())
ashape(a) = a isa Sheep ? :circle : :utriangle
acolor(a) = a isa Sheep ? RGBAf(1.0, 1.0, 1.0, 0.8) : RGBAf(0.2, 0.2, 0.3, 0.8)

# and instruct [`abmplot`](@ref) how to plot grass as a heatmap:
grasscolor(model) = model.countdown ./ model.regrowth_time
# and finally define a colormap for the grass:
heatkwargs = (colormap = [:brown, :green], colorrange = (0, 1))

# and put everything together and give it to [`abmplot`](@ref)
plotkwargs = (;
    agent_color = acolor,
    agent_size = 25,
    agent_marker = ashape,
    offset,
    agentsplotkwargs = (strokewidth = 1.0, strokecolor = :black),
    heatarray = grasscolor,
    heatkwargs = heatkwargs,
)

stable_params = (;
n_sheep = 140,
n_wolves = 3,
dims = (30, 30),
Δenergy_sheep = 5,
sheep_reproduce = 0.31,
wolf_reproduce = 0.06,
Δenergy_wolf = 30,
seed = 71758,
)
sheepwolfgrass = initialize_model(;stable_params...)

abmvideo(
    "sheepwolf.mp4",
    sheepwolfgrass;
    frames = 2000,
    framerate = 8,
    title = "Sheep Wolf Grass",
    plotkwargs...,
)

In [ ]:

sheepwolfgrass = initialize_model()

fig, ax, abmobs = abmplot(sheepwolfgrass; plotkwargs...)
fig

# Now, lets run the simulation and collect some data. Define datacollection:
sheep(a) = a isa Sheep
wolf(a) = a isa Wolf
count_grass(model) = count(model.fully_grown)
# Run simulation:
sheepwolfgrass = initialize_model()
steps = 1000
adata = [(sheep, count), (wolf, count)]
mdata = [count_grass]
adf, mdf = run!(sheepwolfgrass, steps; adata, mdata)

# The following plot shows the population dynamics over time.
# Initially, wolves become extinct because they consume the sheep too quickly.
# The few remaining sheep reproduce and gradually reach an
# equilibrium that can be supported by the amount of available grass.
function plot_population_timeseries(adf, mdf)
    figure = Figure(size = (600, 400))
    ax = figure[1, 1] = Axis(figure; xlabel = "Step", ylabel = "Population")
    sheepl = lines!(ax, adf.time, adf.count_sheep, color = :cornsilk4)
    wolfl = lines!(ax, adf.time, adf.count_wolf, color = RGBAf(0.2, 0.2, 0.3))
    grassl = lines!(ax, mdf.time, mdf.count_grass, color = :green)
    figure[1, 2] = Legend(figure, [sheepl, wolfl, grassl], ["Sheep", "Wolves", "Grass"])
    figure
end

plot_population_timeseries(adf, mdf)
